In [6]:
from database.adatabase import ADatabase
from extractor.alp_client_extractor import ALPClientExtractor
from extractor.tiingo_extractor import TiingoExtractor
from processor.processor import Processor as processor
from datetime import datetime, timedelta
from tqdm import tqdm 
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd
from time import sleep
import matplotlib.pyplot as plt



In [21]:
start = datetime.now() - timedelta(days=2)
extractor = ALPClientExtractor(os.getenv("APCAKEY"),os.getenv("APCASECRET"))
tickers = ["AAVE", "AVAX", "BAT", "BCH", "BTC", "CRV", "DOGE", "DOT", "ETH", "GRT", "LINK", "LTC", "MKR", "SHIB", "SUSHI","USDT","USDC", "UNI", "XTZ", "YFI"]
tickers = [x +"/USD" for x in tickers]
prices = []
for ticker in tickers:
    response = extractor.crypto(ticker,start)
    price = pd.DataFrame(response["bars"][ticker]).rename(columns={"c":"adjclose","t":"date"})[["date","adjclose"]]
    price["date"] = pd.to_datetime(price["date"])
    price = price.sort_values("date")
    price["rolling_return"] = ((price["adjclose"].rolling(30).mean() - price["adjclose"]) / price["adjclose"])
    price["return"] = price["adjclose"].pct_change(5)
    price["avg_return"] = price["return"].rolling(30).mean()
    price["coev"] = (price["adjclose"].rolling(30).mean() / price["adjclose"].rolling(30).std())
    price["ticker"]= ticker
    prices.append(price)

prices = pd.concat(prices).sort_values("date").dropna()

In [30]:
date_range[::5]

[Timestamp('2024-09-23 00:34:00+0000', tz='UTC'),
 Timestamp('2024-09-23 00:39:00+0000', tz='UTC'),
 Timestamp('2024-09-23 00:44:00+0000', tz='UTC'),
 Timestamp('2024-09-23 00:49:00+0000', tz='UTC'),
 Timestamp('2024-09-23 00:54:00+0000', tz='UTC'),
 Timestamp('2024-09-23 00:59:00+0000', tz='UTC'),
 Timestamp('2024-09-23 01:04:00+0000', tz='UTC'),
 Timestamp('2024-09-23 01:09:00+0000', tz='UTC'),
 Timestamp('2024-09-23 01:14:00+0000', tz='UTC'),
 Timestamp('2024-09-23 01:19:00+0000', tz='UTC'),
 Timestamp('2024-09-23 01:24:00+0000', tz='UTC'),
 Timestamp('2024-09-23 01:29:00+0000', tz='UTC'),
 Timestamp('2024-09-23 01:34:00+0000', tz='UTC'),
 Timestamp('2024-09-23 01:39:00+0000', tz='UTC'),
 Timestamp('2024-09-23 01:44:00+0000', tz='UTC'),
 Timestamp('2024-09-23 01:49:00+0000', tz='UTC'),
 Timestamp('2024-09-23 01:54:00+0000', tz='UTC'),
 Timestamp('2024-09-23 01:59:00+0000', tz='UTC'),
 Timestamp('2024-09-23 02:04:00+0000', tz='UTC'),
 Timestamp('2024-09-23 02:09:00+0000', tz='UTC'),


In [23]:
metrics = [
    "rolling_return",
    "return",
    "avg_return"
    ,"coev"
]

In [35]:
date_range = list(prices["date"].unique())

cash = 100
base_position = {"date":date_range[0],"ticker":"","quantity":0,"adjclose":0,"buy_price":0,"buy_date":date_range[0]}
positions = []
analysis = []
for metric in tqdm(metrics):
    for ascending in [True,False]:
        position = base_position.copy()
        for date in date_range:
            try:
                position = position.copy()
                position["date"] = date
                today = prices[prices["date"]==date]
                opportunity = today.sort_values(metric,ascending=ascending).iloc[0]
                if today.index.size >= len(tickers):
                    if position["ticker"] != "":
                        position["adjclose"] = today[today["ticker"]==position["ticker"]]["adjclose"].iloc[0].item()
                        if (date - position["buy_date"]).days > float(5/24*60):
                            cash = position["adjclose"] * position["quantity"]
                            position = base_position.copy()
                            position["date"] = date
                            position["ticker"] = opportunity["ticker"]
                            position["quantity"] = cash * 0.9975 / opportunity["adjclose"] 
                            position["adjclose"] = opportunity["adjclose"]
                            position["buy_price"] = opportunity["adjclose"]
                            position["buy_date"] = date
                    else:
                            position["date"] = date
                            position["ticker"] = opportunity["ticker"]
                            position["quantity"] = cash * 0.9975 / opportunity["adjclose"] 
                            position["adjclose"] = opportunity["adjclose"]
                            position["buy_price"] = opportunity["adjclose"]
                            position["buy_date"] = date
                positions.append(position.copy())
            except Exception as e:
                print(str(e))
        position["metric"] = metric
        position["ascending"] = ascending
        position["pv"] = position["adjclose"] * position["quantity"]
        analysis.append(position)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:17<00:00,  4.31s/it]


In [36]:
a = pd.DataFrame(analysis)
a.sort_values("pv",ascending=False)

,date,ticker,quantity,adjclose,buy_price,buy_date,metric,ascending,pv
1,2024-09-25 19:38:00+00:00,GRT/USD,574.985266,0.180885,0.173918,2024-09-23 07:18:00+00:00,rolling_return,False,104.006210
2,2024-09-25 19:38:00+00:00,GRT/USD,574.985266,0.180885,0.173918,2024-09-23 07:18:00+00:00,return,True,104.006210
6,2024-09-25 19:38:00+00:00,GRT/USD,574.985266,0.180885,0.173918,2024-09-23 07:18:00+00:00,coev,True,104.006210
4,2024-09-25 19:38:00+00:00,CRV/USD,338.919862,0.306720,0.295055,2024-09-23 07:18:00+00:00,avg_return,True,103.953500
7,2024-09-25 19:38:00+00:00,USDT/USD,100.007000,0.999863,0.999930,2024-09-23 07:18:00+00:00,coev,False,99.993300
0,2024-09-25 19:38:00+00:00,AAVE/USD,0.588460,166.150000,169.935000,2024-09-23 07:18:00+00:00,rolling_return,True,97.772678
3,2024-09-25 19:38:00+00:00,AAVE/USD,0.588460,166.150000,169.935000,2024-09-23 07:18:00+00:00,return,False,97.772678
5,2024-09-25 19:38:00+00:00,AAVE/USD,0.588460,166.150000,169.935000,2024-09-23 07:18:00+00:00,avg_return,False,97.772678


In [37]:
today

,date,adjclose,rolling_return,return,avg_return,coev,ticker
2065,2024-09-25 19:38:00+00:00,6.837000,0.003745,-0.002528,-5.669500e-04,613.534438,UNI/USD
2127,2024-09-25 19:38:00+00:00,2580.630000,0.000504,0.000668,5.813421e-05,1375.059116,ETH/USD
3785,2024-09-25 19:38:00+00:00,0.000015,0.002530,0.000323,-7.263079e-04,327.413286,SHIB/USD
3042,2024-09-25 19:38:00+00:00,0.707557,0.001657,-0.001568,2.206017e-04,947.289065,SUSHI/USD
3450,2024-09-25 19:38:00+00:00,0.999873,0.000007,-0.000008,-1.723406e-07,11637.440131,USDT/USD
3816,2024-09-25 19:38:00+00:00,0.999960,-0.000018,0.000236,-1.036486e-06,9416.455698,USDC/USD
1622,2024-09-25 19:38:00+00:00,67.147000,0.002071,-0.000551,-4.899001e-05,575.119890,LTC/USD
3632,2024-09-25 19:38:00+00:00,0.180285,0.003843,-0.001548,-9.011575e-04,430.571418,GRT/USD
